# Market Clearing Price Model

In [3]:

# Imports
using CSV, DataFrames, JuMP, Gurobi, Plots, Formatting, LinearAlgebra, Random

In [12]:
# Define a mutable struct to represent a node with the required properties
mutable struct Node
    area::String  # The area is now a string, which will be randomly selected from "A1", "A2", "A3"
    x_coordinate::Float64
    y_coordinate::Float64
    exists::Bool
    comment::Union{String, Nothing}
end

In [13]:
# Function to generate random nodes
function generate_nodes(N::Int)
    areas = ["A1", "A2", "A3"]  # Define the potential areas
    
    nodes = Vector{Node}(undef, N)  # Create an array to hold Node structs
    
    for i in 1:N
        # Randomly select an area
        area = rand(areas)
        # Randomly generate values for the other node attributes
        x_coordinate = rand(0.0:0.01:10.0)  # Random x-coordinate between 0 and 10
        y_coordinate = rand(0.0:0.01:10.0)  # Random y-coordinate between 0 and 10
        exists = rand(Bool)  # Randomly decide if the node exists
        comment = rand(Bool) ? "Active node" : nothing  # Optionally assign a comment
        
        # Create a new node with the generated attributes
        nodes[i] = Node(area, x_coordinate, y_coordinate, exists, comment)
    end
    return nodes
end

generate_nodes (generic function with 1 method)

In [19]:

# Function to display the nodes in a DataFrame for easy viewing
function nodes_to_dataframe(nodes::Vector{Node})
    df = DataFrame(
        area = [node.area for node in nodes],
        x_coordinate = [node.x_coordinate for node in nodes],
        y_coordinate = [node.y_coordinate for node in nodes],
        exists = [node.exists for node in nodes],
        comment = [node.comment for node in nodes]
    )
    return df
end


nodes_to_dataframe (generic function with 1 method)

In [ ]:
# Function to calculate the distance matrix between nodes
function calculate_distance_matrix(nodes::Vector{Node})
    N = length(nodes)
    distance_matrix = zeros(Float64, N, N)  # Initialize an N x N matrix of zeros
    
    for i in 1:N
        for j in 1:N
            if i != j  # Avoid self-distances
                # Calculate Euclidean distance between node i and node j
                xi, yi = nodes[i].x_coordinate, nodes[i].y_coordinate
                xj, yj = nodes[j].x_coordinate, nodes[j].y_coordinate
                distance_matrix[i, j] = sqrt((xj - xi)^2 + (yj - yi)^2)
            else
                distance_matrix[i, j] = 0.0  # Distance to itself is zero
            end
        end
    end
    return distance_matrix
end

In [15]:
# Example usage
N = 10  # Number of nodes
nodes = generate_nodes(N)
df = display_nodes(nodes)

# Show the DataFrame
println(df)

10×5 DataFrame
 Row │ area    x_coordinate  y_coordinate  exists  comment     
     │ String  Float64       Float64       Bool    Union…      
─────┼─────────────────────────────────────────────────────────
   1 │ A3              2.01          5.76    true  Active node
   2 │ A3              3.77          8.23    true              
   3 │ A3              9.84          1.94    true  Active node
   4 │ A1              9.99          6.86   false              
   5 │ A1              6.74          7.87    true  Active node
   6 │ A2              9.78          7.63    true  Active node
   7 │ A2              6.47          4.26    true  Active node
   8 │ A2              3.71          7.05   false  Active node
   9 │ A1              2.86          9.35    true  Active node
  10 │ A3              9.28          2.88    true              


In [18]:
println(df[!, "area"])
println

["A3", "A3", "A3", "A1", "A1", "A2", "A2", "A2", "A1", "A3"]


In [11]:

# Define the number of nodes (N)
N = 10  # You can change this to any number

# Generate random Cartesian coordinates between 0 and 10 for N nodes
coordinates = rand(0:0.01:10, N, 2)

# Display the generated coordinates
println("Generated coordinates for $N nodes:")
println(coordinates[:,1])
println(coordinates)
println(typeof(coordinates))


Generated coordinates for 10 nodes:
[5.84, 1.03, 6.37, 9.18, 2.65, 3.95, 0.71, 0.48, 2.87, 9.75]
[5.84 2.69; 1.03 4.08; 6.37 1.55; 9.18 0.87; 2.65 9.81; 3.95 5.4; 0.71 3.45; 0.48 1.4; 2.87 7.37; 9.75 8.24]
Matrix{Float64}
